In [42]:
%matplotlib inline
import matplotlib
import seaborn as sns

import numpy as np
import pandas as pd

import ujson as json
import gzip, itertools, numbers

from collections import defaultdict

from sklearn import base, utils

from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, RANSACRegressor, ElasticNet, Ridge, LogisticRegression

from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

from sklearn import model_selection
from sklearn.model_selection import GridSearchCV

# ML: Predicting Star Ratings

Our objective is to predict a new venue's popularity from information available when the venue opens.  We will do this by machine learning from a dataset of venue popularities provided by Yelp.  The dataset contains meta data about the venue (where it is located, the type of food served, etc.).  It also contains a star rating. Note that the venues are not limited to restaurants.

In [2]:
# TODO install aws-packages and find an alternative data-source
#!aws s3 sync s3://dataincubator-course/mldata/ . --exclude '*' --include 'yelp_train_academic_dataset_business.json.gz'

The training data are a series of JSON objects, in a gzipped file. 

In [3]:
with gzip.open('yelp_train_academic_dataset_business.json.gz') as f:
    data = [json.loads(line) for line in f]
    
star_ratings = [row['stars'] for row in data]

X, Y = utils.shuffle(data, star_ratings, random_state=41)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=42, stratify = star_ratings)

# 1. city_avg
The venues belong to different cities. You can image that the ratings in some cities are probably higher than others. We wish to build an estimator to make a prediction based on this, but first we need to work out the average rating for each city.

In [4]:
class CityEstimator(base.BaseEstimator, base.RegressorMixin):
    
    def __init__(self):
        self.avg_stars = defaultdict(int)
    
    def fit(self, X, y):
        star_sum = defaultdict(int)
        count = defaultdict(int)

        for row, stars in zip(X, y):
            star_sum[row['city']] += stars
            count[row['city']] += 1    
        for k in star_sum:
            self.avg_stars[k] = star_sum[k]/float(count[k]) 
        
        return self
    
    def predict(self, X):
        return [self.avg_stars[row['city']] for row in X] # X has to be a pd.DataFrame or a Dictionary !!!!
        #return [ self.avg_stars[row['city']] if row['city'] in self.avg_stars else np.nan for row in X ]

city_avg = CityEstimator()
city_avg.fit(X_train, Y_train)

CityEstimator()

# 2. lat_long_model
You can imagine that a city-based model might not be sufficiently fine-grained. For example, we know that some neighborhoods are trendier than others.  Use the latitude and longitude of a venue as features that help you understand neighborhood dynamics.

In [5]:
class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    '''
    extract selected key-value pairs from a dictionary and transform them to an array
    '''
    def __init__(self, col_names):
        self.col_names = col_names
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X): # every row of X is a dictionary
        data = [ [row[col_name] for col_name in self.col_names] for row in X ]
        return data 

**Use k_nn**

In [7]:
knn_pipe = Pipeline([
        ('my_cst',ColumnSelectTransformer(['latitude', 'longitude'])), 
        ('my_knn',KNeighborsRegressor(n_neighbors=5)) 
        ])

expres_cv = model_selection.ShuffleSplit(n_splits=3, test_size=0.2, random_state=41)

parameters = {'my_knn__n_neighbors': range(34,55,4)}

gs_latlong_knn = GridSearchCV(knn_pipe, param_grid=parameters, cv=expres_cv, scoring='neg_mean_squared_error', n_jobs=1) 

gs_latlong_knn.fit(X_train, Y_train)

print(gs_latlong_knn.best_params_)
print(gs_latlong_knn.best_score_)

Y_train_predict = gs_latlong_knn.best_estimator_.predict(X_train)
Y_test_predict  = gs_latlong_knn.best_estimator_.predict(X_test )

print 'r2_score_train = %.2f r2_score_test = %.2f'\
            %( float(r2_score(Y_train, Y_train_predict)), float(r2_score(Y_test, Y_test_predict)))

{'my_knn__n_neighbors': 54}
-0.777439799843
r2_score_train = 0.05 r2_score_test = 0.03


# 3. category_model
While location is important, we could also try seeing how predictive the venue's category is. Build an estimator that considers only the categories.

The categories come as a list of strings, but the built-in estimators all need numeric input.  The standard way to deal with categorical features is **one-hot encoding**, also known as dummy variables.  In this approach, each category gets its own column in the feature matrix. If the row has a given category, that column gets filled with a 1.  Otherwise, it is 0.

The `ColumnSelectTransformer` from the previous question can be used to extract the categories column as a list of strings.  Scikit Learn provides [DictVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html#sklearn.feature_extraction.DictVectorizer), which takes in a list of dictionaries.  It creates a column in the output matrix for each key in the dictionary and fills it with the value associated with it.  Missing keys are filled with zeros.  Therefore, we need only build a transformer that takes a list strings to a dictionary with keys given by those strings and values one.

In [11]:
class DictEncoder(base.BaseEstimator, base.TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):  # this will work only if I pass to it the 'category' column/key
        return [ { el:1 for el in col} for row in X for col in row ]
    

category_pipe = Pipeline([
        ('my_col_select', ColumnSelectTransformer(['categories'])), 
        ('my_Dict_enc', DictEncoder()),
        ('my_Dict_vect', DictVectorizer(sparse=False) ),
        ('my_Ridge', Ridge(alpha=0.1)) 
        ])


parameters = {'my_Ridge__alpha': np.logspace(-1, 2, 7) }
gs_category = GridSearchCV(category_pipe, parameters, cv=3, n_jobs=1) #scoring='neg_mean_squared_error', 
gs_category.fit(X_train, Y_train)        
        
print(gs_category.best_params_)
print(gs_category.best_score_)

gs_category.best_estimator_

Y_train_predict = gs_category.best_estimator_.predict(X_train)
Y_test_predict  = gs_category.best_estimator_.predict(X_test )

print 'alpha = %.4f r2_score_train = %.2f r2_score_test = %.2f'\
            %(alpha, float(r2_score(Y_train, Y_train_predict)), float(r2_score(Y_test, Y_test_predict)))

{'my_Ridge__alpha': 10.0}
0.171877621576
alpha = 1000.0000 r2_score_train = 0.20 r2_score_test = 0.17


# 4. attribute_model
There is even more information in the attributes for each venue.  Let's build an estimator based on these.

Venues attributes may be nested:
```
{
  'Attire': 'casual',
  'Accepts Credit Cards': True,
  'Ambiance': {'casual': False, 'classy': False}
}
```
We wish to encode them with one-hot encoding.  The `DictVectorizer` can do this, but only once we've flattened the dictionary to a single level, like so:
```
{
  'Attire_casual' : 1,
  'Accepts Credit Cards': 1,
  'Ambiance_casual': 0,
  'Ambiance_classy': 0
}
```

Build a custom transformer that flattens the attributes dictionary.  Place this in a pipeline with a `DictVectorizer` and a regressor.

You may find it difficult to find a single regressor that does well enough.  A common solution is to use a linear model to fit the linear part of some data, and use a non-linear model to fit the residual that the linear model can't fit.  Build a residual estimator that takes as an argument two other estimators.  It should use the first to fit the raw data and the second to fit the residuals of the first.

In [12]:
#################################################################### feed with one feature that contains
class DictEncoder(base.BaseEstimator, base.TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):  # this will work only if I pass to it the 'category' column/key
        return [ { el:1 for el in col} for row in X for col in row ]

#################################################################### feed with ColumnSelectTransformer(['attributes'])
class Attrib_Flatten(base.BaseEstimator, base.TransformerMixin):
    def fit(self, X, y=None):
        return self    
    def transform(self, X): 
        return [ { k1:v1 for k,v in col.iteritems() for k1,v1 in Dict_Maker( k, v ).iteritems() } for row in X for col in row]
        
def Dict_Maker( k, v ):          # returns {k1:v1, k2:v2, .... , kn:vn}
    if isinstance(v, basestring):
        return { k + '_' + v : 1}
    if isinstance(v, bool):
        return { k : int(v==True)} 
    if isinstance(v, numbers.Number):
        return { k : v}
    if isinstance(v, dict):
        return { k2:v2      for k1,v1 in v.iteritems() for k2,v2 in Dict_Maker( k + '_' + k1, v1 ).iteritems() }
    if isinstance(v, list):
        return { k2:v2      for v1 in v for k2,v2 in Dict_Maker(k,v1).iteritems() } # read additional footnotes [1]
    else:
        return {}

# [1] in this case we assume that  all elements in v are strings or dictionaries
# in the other cases the algotithm wont brake but it will produce less Key-Value pairs
# Example:                k:[v1, v2, v3] -> k:v3 if v1, v2, v3 are integers

### 4.1 Use Logistic Regression
The ratings can take only the values [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]

In [18]:
import copy
    
class another_Y_trf(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self):
        self.my_map = {} # we could also use defaultdict ...
        self.my_inv_map = {}
    
    def fit(self, Y):
        for idx, el in enumerate( list(set(Y)) ):
            self.my_map[el]        = idx+1
            self.my_inv_map[idx+1] = el
        return self
    
    def transform(self, Y):
        return np.array([ self.my_map[el] for el in Y ])
    
    def inv_trf(self, Y_trf):
        return np.array([ self.my_inv_map[idx] for idx in Y_trf ])
    
    
    
pipe_attributes_LogR = Pipeline([
    ('my_cst', ColumnSelectTransformer(['attributes']) ), 
    ('my_At_Flat', Attrib_Flatten() ),
    ('my_DVect', DictVectorizer(sparse=False) ),
    ('my_Logistic', LogisticRegression(C=0.1, n_jobs=-1, max_iter=100, solver='sag'))
    ])

my_Y_trf = another_Y_trf()
my_Y_trf.fit(Y_train)

YY_train = my_Y_trf.transform(Y_train)
YY_test  = my_Y_trf.transform(Y_test )

for C in np.logspace(-4,-1,4):
    pipe_attributes_LogR.set_params( my_Logistic__C=C )
    
    pipe_attributes_LogR.fit(X_train, YY_train)
    
    Y_train_predict = my_Y_trf.inv_trf( pipe_attributes_LogR.predict(X_train) )
    Y_test_predict  = my_Y_trf.inv_trf( pipe_attributes_LogR.predict(X_test ) )

    print 'C = %.4f r2_score_train = %.2f r2_score_test = %.2f'\
            %(C, float(r2_score(Y_train, Y_train_predict)), float(r2_score(Y_test, Y_test_predict)))

C = 0.0001 r2_score_train = -0.06 r2_score_test = -0.06
C = 0.0010 r2_score_train = -0.59 r2_score_test = -0.61
C = 0.0100 r2_score_train = -0.59 r2_score_test = -0.60
C = 0.1000 r2_score_train = -0.59 r2_score_test = -0.62


### 4.2 Use Linear regression

In [31]:
pipe_attributes_LinR = Pipeline([
    ('my_cst', ColumnSelectTransformer(['attributes']) ), 
    ('my_At_Flat', Attrib_Flatten() ),
    ('my_DVect', DictVectorizer(sparse=False) ),
    #('my_PCA', PCA(n_components=n)),
    ('my_Ridge', Ridge(alpha=1.0))
    ])

parameters = {'my_Ridge__alpha': np.logspace(-3, 2, 6) }
gs_attribute_LinR = GridSearchCV(pipe_attributes_LinR, parameters, cv=3, n_jobs=1)
gs_attribute_LinR.fit(X_train, Y_train)        
        
print(gs_attribute_LinR.best_params_)
print(gs_attribute_LinR.best_score_)


Y_train_predict = gs_attribute_LinR.best_estimator_.predict(X_train)
Y_test_predict  = gs_attribute_LinR.best_estimator_.predict(X_test )

print 'alpha = %.4f r2_score_train = %.2f r2_score_test = %.2f'\
            %(alpha, float(r2_score(Y_train, Y_train_predict)), float(r2_score(Y_test, Y_test_predict)))

{'my_Ridge__alpha': 10.0}
0.0704371250279
alpha = 100.0000 r2_score_train = 0.07 r2_score_test = 0.07


### 4.3 Use RandomForest

In [34]:
pipe_attributes_RF = Pipeline([ 
    ('my_cst', ColumnSelectTransformer(['attributes']) ), 
    ('my_At_Flat', Attrib_Flatten() ),
    ('my_DVect', DictVectorizer(sparse=False) ),
    ('my_RF', RandomForestRegressor(n_jobs = 1, max_depth=10))
    ])


parameters = {'my_RF__n_estimators': [20, 50, 100], 
              'my_RF__max_depth': [2, 6, 10]
             }


gs_attribute_RF = GridSearchCV(pipe_attributes_RF, parameters, cv=3, n_jobs=1)
gs_attribute_RF.fit(X_train, Y_train)        
        
print(gs_attribute_RF.best_params_)
print(gs_attribute_RF.best_score_)


Y_train_predict = gs_attribute_RF.best_estimator_.predict(X_train)
Y_test_predict  = gs_attribute_RF.best_estimator_.predict(X_test )

print 'alpha = %.4f r2_score_train = %.2f r2_score_test = %.2f'\
            %(alpha, float(r2_score(Y_train, Y_train_predict)), float(r2_score(Y_test, Y_test_predict)))

{'my_RF__n_estimators': 100, 'my_RF__max_depth': 10}
0.0831967040806
alpha = 100.0000 r2_score_train = 0.12 r2_score_test = 0.09


### 4.4 Use k_nn

In [45]:
pipe_attributes_knn = Pipeline([ 
    ('my_cst', ColumnSelectTransformer(['attributes']) ), 
    ('my_At_Flat', Attrib_Flatten() ),
    ('my_DVect', DictVectorizer(sparse=False) ),
    ('my_svd', TruncatedSVD(n_components=30) ),
    ('my_knn', KNeighborsRegressor(n_neighbors=5) )
    ])


parameters = {'my_knn__n_neighbors': [20, 30, 40, 50]}

gs_attribute_knn = GridSearchCV( pipe_attributes_knn, param_grid = parameters, cv=3, n_jobs=1)
gs_attribute_knn.fit(X_train, Y_train) 
        
print(gs_attribute_knn.best_params_)
print(gs_attribute_knn.best_score_)


Y_train_predict = gs_attribute_knn.best_estimator_.predict(X_train)
Y_test_predict  = gs_attribute_knn.best_estimator_.predict(X_test )

print 'alpha = %.4f r2_score_train = %.2f r2_score_test = %.2f'\
            %(alpha, float(r2_score(Y_train, Y_train_predict)), float(r2_score(Y_test, Y_test_predict)))

{'my_knn__n_neighbors': 50}
0.0654473572302
alpha = 100.0000 r2_score_train = 0.08 r2_score_test = 0.07


### 4.5 Create a pipeline containing linear and nonlinear models where the nonlinear model try to decrease the residua from the linear model 

In [35]:
class EnsembleTransformer(base.BaseEstimator, base.TransformerMixin):    
    
    def __init__(self, base_estimator, residual_estimator_1, residual_estimator_2):
        self.base_estimator = base_estimator
        self.residual_estimator_1 = residual_estimator_1
        self.residual_estimator_2 = residual_estimator_2
        
    def fit(self, X, y):
        self.base_estimator.fit(X, y)
        y_err = y - self.base_estimator.predict(X)
        
        self.residual_estimator_1.fit(X, y_err)
        self.residual_estimator_2.fit(X, y_err)
        
        return self
    
    def transform(self, X):
        all_ests = [ self.base_estimator, self.residual_estimator_1, self.residual_estimator_2 ]
        return np.array([est.predict(X) for est in all_ests]).T

In [ ]:
ensemble_pipe = Pipeline([
        ('ensemble', EnsembleTransformer(base_estimator = gs_attribute_LinR.best_estimator_, 
                                         residual_estimator_1 = gs_attribute_RF.best_estimator_, 
                                         residual_estimator_2 = gs_attribute_knn.best_estimator_  ) ),
        ('blend', Ridge(alpha=0))
        ])

parameters = {'my_knn__n_neighbors': range(10,59,4) }


gs_attribute_ensemble = GridSearchCV( ensemble_pipe, param_grid = parameters, cv=3, n_jobs=1)
gs_attribute_ensemble.fit(X_train, Y_train)    
        
print(gs_attribute_ensemble.best_params_)
print(gs_attribute_ensemble.best_score_ )


Y_train_predict = gs_attribute_ensemble.best_estimator_.predict(X_train)
Y_test_predict  = gs_attribute_ensemble.best_estimator_.predict(X_test )

print 'alpha = %.4f r2_score_train = %.2f r2_score_test = %.2f'\
            %(alpha, float(r2_score(Y_train, Y_train_predict)), float(r2_score(Y_test, Y_test_predict)))

# 5. full_model
So far we have only built models based on individual features.  Now we will build an ensemble regressor that averages together the estimates of the four previous regressors.

In order to use the existing models as input to an estimator, we will have to turn them into transformers.  (A pipeline can contain at most a single estimator.)  Build a custom `ModelTransformer` class that takes an estimator as an argument.  When `fit()` is called, the estimator should be fit.  When `transform()` is called, the estimator's `predict()` method should be called, and its results returned.

Note that the output of the `transform()` method should be a 2-D array with a single column, in order for it to work well with the Scikit Learn pipeline.  If you're using Numpy arrays, you can use `.reshape(-1, 1)` to create a column vector.  If you are just using Python lists, you will want a list of lists of single elements.

In [ ]:
class EstimatorTransformer(base.BaseEstimator, base.TransformerMixin):

    def __init__(self, estimator):
        self.estimator = estimator 
    
    def fit(self, X, y):
        self.estimator.fit(X, y)
        return self
    
    def transform(self, X):
        return np.array(   self.estimator.predict(X)  ).reshape(-1, 1)

In [ ]:
# change ColumnSelectTransform to return np.array instead of array
from sklearn.pipeline import FeatureUnion

pipe_LL = Pipeline([('my_cst',ColumnSelectTransformer(['latitude', 'longitude'])), 
                    ('my_knn',KNeighborsRegressor(n_neighbors=40)) ])

pipe_Cateory = Pipeline([('my_cst', ColumnSelectTransformer(['categories'])), 
                         ('my_Denc', DictEncoder()),
                         ('my_DVect', DictVectorizer(sparse=False) ),
                         ('my_Ridge', Ridge(alpha=6)) ])

pipe_Attributes = Pipeline([('my_cst', ColumnSelectTransformer(['attributes']) ), 
                            ('my_At_Flat', Attrib_Flatten() ),
                            ('my_DVect', DictVectorizer(sparse=False) ),
                            ('my_RF', ensemble.RandomForestRegressor(max_features=20, n_estimators=500, min_samples_leaf=4))])


ET_CityAvg    = EstimatorTransformer(city_avg)

ET_LatLong    = EstimatorTransformer(gs_latlong_knn.best_estimator_)

ET_Cateory    = EstimatorTransformer(gs_category.best_estimator_)

ET_Attributes = EstimatorTransformer(gs_attribute_ensemble.best_estimator_)

union = FeatureUnion([ ('ET_CityAvg',    ET_CityAvg),  
                       ('ET_LatLong',    ET_LatLong), 
                       ('ET_Cateory',    ET_Cateory), 
                       ('ET_Attributes', ET_Attributes) ])

Finally, use a pipeline to combine the feature union with a linear regression (or another model) to weight the predictions.

In [ ]:
ensemble_pipe = Pipeline([
        ('ensemble', union),
        ('blend', Ridge(alpha=0))
        ])

ensemble_pipe.fit(X_train, Y_train)

Y_train_predict = ensemble_pipe.best_estimator_.predict(X_train)
Y_test_predict  = ensemble_pipe.best_estimator_.predict(X_test )

print 'alpha = %.4f r2_score_train = %.2f r2_score_test = %.2f'\
            %(alpha, float(r2_score(Y_train, Y_train_predict)), float(r2_score(Y_test, Y_test_predict)))